In [1]:
import os
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy import stats
from sklearn.preprocessing import StandardScaler
from textwrap import wrap
from IPython.display import display, HTML
import mat4py
import scipy.io
from itertools import chain
from scipy.stats import gamma
from scipy.stats import spearmanr
import statistics
import random 
import math
from scipy.stats import zscore
from pandas.api.types import CategoricalDtype
import scipy.io as spio

In [2]:
def grab_FD(data_directory):
    '''
    Loads in all data files from the data folder and adds a colunm called particpant with ID.
    :param a: list of file names
    :return: a raw df for all participants
    '''
    
    data_files = os.listdir(data_directory)
    within_FD = []
    
    for file in data_files:
        participants = file[0:6]
        run = file[-27:-26]
        print(participants,run)
        
        df = pd.read_csv(data_directory + '/' + file, delimiter=' ', header=None)
        df['participant'] = pd.Series(participants, index=df.index)
        df['run'] = pd.Series(run, index=df.index)
        within_FD.append(df)

    return within_FD

# 1. Get trial to trial motion information

In [3]:
##read in demean files
data_directory = '/Users/ziweizhang/Downloads/HCP37T_LL/data/brain/compiled_motion'
data_files = os.listdir(data_directory)
# print(data_files[0],
# data_files[0][0:6],
#       data_files[0][-27:-26]
#      )


In [6]:
within_FD = pd.concat(grab_FD(data_directory),ignore_index=True)
within_FD = within_FD.rename(columns={0: 'x', 1: 'y', 2: 'z', 3: 'p', 4: 'r', 5:'ya',
                                     6: 'x_d', 7: 'y_d', 8: 'z_d', 9: 'p_d', 10: 'r_d', 11:'ya_d'})


725751 1
818859 3
102311 1
131217 3
180533 2
601127 2
765864 2
389357 3
146735 2
181232 1
671855 1
573249 4
198653 2
137128 3
182739 2
195041 3
178243 4
105923 1
872764 2
130114 1
196144 3
246133 1
393247 3
406836 1
200614 3
191033 3
395756 1
263436 1
177645 1
192641 2
137128 4
573249 3
360030 1
389357 4
397760 2
135124 2
818859 4
131217 4
393247 4
191033 4
200614 4
463040 1
146937 2
104416 2
164636 2
196144 4
178243 3
541943 2
187345 1
617748 1
525541 1
195041 4
627549 3
159239 4
155938 1
826353 3
193845 2
429040 1
757764 1
209228 3
690152 2
467351 1
185442 1
172130 2
789373 1
203418 3
901139 1
199655 2
146129 1
108323 1
204521 2
927359 2
125525 1
165436 3
177140 3
814649 4
191336 1
352738 1
783462 4
330324 1
173334 2
146432 1
209228 4
724446 2
899885 1
770352 1
971160 2
943862 1
385046 1
942658 2
826353 4
732243 1
627549 4
159239 3
898176 2
783462 3
167440 2
436845 1
878877 1
175237 1
115017 1
165436 4
814649 3
177140 4
176542 2
148133 1
111514 1
203418 4
156334 1
901139 3
214524 2
1

In [7]:
within_FD

,x,y,z,p,r,ya,x_d,y_d,z_d,p_d,r_d,ya_d,participant,run
0,0.000000,0.138005,0.118325,-0.031971,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,725751,1
1,0.006305,0.061841,0.139283,-0.028992,0.000000,-0.015069,0.006305,-0.076164,0.020958,0.002979,0.000000,-0.015069,725751,1
2,0.003242,0.064949,0.183913,-0.021887,0.000000,-0.023090,-0.003063,0.003108,0.044630,0.007105,0.000000,-0.008021,725751,1
3,-0.000841,0.307370,0.193098,-0.043258,0.009626,-0.040623,-0.004083,0.242421,0.009185,-0.021371,0.009626,-0.017533,725751,1
4,-0.003224,0.201727,0.141591,-0.024293,0.005844,-0.037701,-0.002383,-0.105643,-0.051507,0.018965,-0.003782,0.002922,725751,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434940,-0.038154,0.630233,1.653120,0.442724,0.133671,-0.328763,0.004440,0.117847,-0.012901,-0.053171,-0.016272,-0.016157,192641,4
434941,-0.040655,0.490082,1.645740,0.471430,0.139401,-0.313981,-0.002501,-0.140151,-0.007380,0.028706,0.005730,0.014782,192641,4
434942,-0.042837,0.513701,1.665160,0.515032,0.143698,-0.328935,-0.002182,0.023619,0.019425,0.043602,0.004297,-0.014954,192641,4
434943,-0.041858,0.644456,1.669390,0.445303,0.138427,-0.329222,0.000979,0.130755,0.004223,-0.069729,-0.005271,-0.000287,192641,4


In [11]:
subs_brain_score_order = ['102311', '104416', '105923', '108323', '109123', '111514', '114823', '115017', 
        '118225', '125525', '128935', '130114', '131217', '135124', '137128', '146129', 
        '146432', '146735', '146937', '148133', '155938', '156334', '157336', '158035', 
        '159239', '162935', '164636', '165436', '167036', '167440', '172130', '173334', 
        '175237', '176542', '177140', '177645', '178243', '180533', '181232', '182739', 
        '185442', '187345', '191033', '191336', '191841', '192641', '193845', '195041', 
        '196144', '198653', '199655', '200311', '200614', '203418', '204521', '209228', 
        '212419', '214019', '214524', '239136', '246133', '249947', '251833', '263436', 
        '318637', '330324', '352738', '360030', '380036', '385046', '389357', '393247', 
        '395756', '397760', '401422', '406836', '412528', '429040', '436845', '463040', 
        '467351', '525541', '541943', '573249', '581450', '601127', '617748', '627549', 
        '638049', '654552', '671855', '680957', '690152', '724446', '725751', '732243', 
        '757764', '765864', '770352', '782561', '783462', '789373', '814649', '818859', 
        '826353', '859671', '861456', '872764', '878877', '898176', '899885', '901139', 
        '910241', '927359', '942658', '943862', '958976', '966975', '971160']

In [12]:
subs_brain_motion_order = within_FD.participant.unique()
subs_brain_motion_reorder = CategoricalDtype(subs_brain_score_order, ordered=True)
subs_brain_motion_reorder

CategoricalDtype(categories=['102311', '104416', '105923', '108323', '109123', '111514',
                  '114823', '115017', '118225', '125525',
                  ...
                  '898176', '899885', '901139', '910241', '927359', '942658',
                  '943862', '958976', '966975', '971160'],
                 ordered=True)

In [13]:
within_FD_ordered = within_FD.copy()

In [14]:
within_FD_ordered['participant'] = within_FD_ordered['participant'].astype(subs_brain_motion_reorder)
within_FD_ordered['participant']


0         725751
1         725751
2         725751
3         725751
4         725751
           ...  
434940    192641
434941    192641
434942    192641
434943    192641
434944    192641
Name: participant, Length: 434945, dtype: category
Categories (119, object): ['102311' < '104416' < '105923' < '108323' ... '943862' < '958976' < '966975' < '971160']

In [15]:
within_FD_ordered = within_FD_ordered.sort_values(['participant','run']).reset_index(drop=True)
within_FD_ordered


,x,y,z,p,r,ya,x_d,y_d,z_d,p_d,r_d,ya_d,participant,run
0,0.000000,0.040730,0.070119,-0.046352,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,102311,1
1,0.000000,0.006030,0.047756,0.000000,0.000000,0.000000,0.000000,-0.034700,-0.022363,0.046352,0.000000,0.000000,102311,1
2,0.000000,0.014362,0.078550,-0.021085,0.000000,0.000000,0.000000,0.008332,0.030794,-0.021085,0.000000,0.000000,102311,1
3,0.000007,0.089937,0.148111,-0.098491,0.000000,0.000000,0.000007,0.075575,0.069561,-0.077406,0.000000,0.000000,102311,1
4,0.000000,0.045510,0.111452,-0.077235,0.000000,0.000000,-0.000007,-0.044427,-0.036659,0.021256,0.000000,0.000000,102311,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434940,0.085961,-0.149918,-0.415211,0.237434,-0.267972,0.056666,-0.022449,-0.026170,-0.024769,0.047728,0.057984,0.009225,971160,4
434941,0.075893,-0.066543,-0.401432,0.214286,-0.224370,0.069099,-0.010068,0.083375,0.013779,-0.023148,0.043602,0.012433,971160,4
434942,0.071771,-0.027297,-0.384546,0.219557,-0.213255,0.060218,-0.004122,0.039246,0.016886,0.005271,0.011115,-0.008881,971160,4
434943,0.074034,0.025249,-0.382854,0.219557,-0.222823,0.071161,0.002263,0.052546,0.001692,0.000000,-0.009568,0.010943,971160,4


In [16]:
FD_summary = pd.DataFrame(within_FD_ordered.groupby(['participant','run']).count().reset_index(drop=True))
#display(HTML(FD_summary.to_html()))
FD_summary


,x,y,z,p,r,ya,x_d,y_d,z_d,p_d,r_d,ya_d
0,921,921,921,921,921,921,921,921,921,921,921,921
1,918,918,918,918,918,918,918,918,918,918,918,918
2,915,915,915,915,915,915,915,915,915,915,915,915
3,901,901,901,901,901,901,901,901,901,901,901,901
4,921,921,921,921,921,921,921,921,921,921,921,921
...,...,...,...,...,...,...,...,...,...,...,...,...
471,901,901,901,901,901,901,901,901,901,901,901,901
472,921,921,921,921,921,921,921,921,921,921,921,921
473,918,918,918,918,918,918,918,918,918,918,918,918
474,915,915,915,915,915,915,915,915,915,915,915,915


In [17]:
# save frame-to-frame displacement in raw data order
#within_FD_ordered.to_csv("/Users/ziweizhang/Downloads/HCP37T_LL/output/motion/within_FD.csv",index=False)


# Calculate FD

In [14]:
# https://wiki.cam.ac.uk/bmuwiki/FMRI
# FD(t)=sum{|d(t-1)-d(t)|} + 50 * (pi /180) * sum{|r(t-1)-r(t)|} where d denotes translation distances {x,y,z}, and r denotes rotation angles {α,β,γ}. 
# For each participant, a single (scalar) estimate of overall motion, the mean FD, can be calculated by averaging the FD time series.

# clip = ["1_clip1","1_clip2","1_clip3","1_clip4","2_clip1","2_clip2","2_clip3",
#         "2_test_retest","3_clip1","3_clip2","3_clip3","3_clip4","4_clip1","4_clip2",
#         "4_clip3"]


In [36]:
within_FD_ordered["run"] = pd.to_numeric(within_FD_ordered["run"])
within_FD_ordered

,x,y,z,p,r,ya,x_d,y_d,z_d,p_d,r_d,ya_d,participant,run
0,0.000000,0.040730,0.070119,-0.046352,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,102311,1
1,0.000000,0.006030,0.047756,0.000000,0.000000,0.000000,0.000000,-0.034700,-0.022363,0.046352,0.000000,0.000000,102311,1
2,0.000000,0.014362,0.078550,-0.021085,0.000000,0.000000,0.000000,0.008332,0.030794,-0.021085,0.000000,0.000000,102311,1
3,0.000007,0.089937,0.148111,-0.098491,0.000000,0.000000,0.000007,0.075575,0.069561,-0.077406,0.000000,0.000000,102311,1
4,0.000000,0.045510,0.111452,-0.077235,0.000000,0.000000,-0.000007,-0.044427,-0.036659,0.021256,0.000000,0.000000,102311,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434940,0.085961,-0.149918,-0.415211,0.237434,-0.267972,0.056666,-0.022449,-0.026170,-0.024769,0.047728,0.057984,0.009225,971160,4
434941,0.075893,-0.066543,-0.401432,0.214286,-0.224370,0.069099,-0.010068,0.083375,0.013779,-0.023148,0.043602,0.012433,971160,4
434942,0.071771,-0.027297,-0.384546,0.219557,-0.213255,0.060218,-0.004122,0.039246,0.016886,0.005271,0.011115,-0.008881,971160,4
434943,0.074034,0.025249,-0.382854,0.219557,-0.222823,0.071161,0.002263,0.052546,0.001692,0.000000,-0.009568,0.010943,971160,4


In [43]:
runs = [1,2,3,4]

appended_data = []
for s in subs_brain_score_order:
    for r in runs:
        df_sub_run = within_FD_ordered.loc[(within_FD_ordered['participant'] == s) & (within_FD_ordered['run'] == r), ['x','y','z','p','r','ya']]
        
        df_sub_run['participant_ID'] = [s] * len(df_sub_run)
        df_sub_run['run'] = [r] * len(df_sub_run)
        df_sub_run['x_diff'] = abs(df_sub_run['x'].shift() - df_sub_run['x'])
        df_sub_run['y_diff'] = abs(df_sub_run['y'].shift() - df_sub_run['y'])
        df_sub_run['z_diff'] = abs(df_sub_run['z'].shift() - df_sub_run['z'])
        df_sub_run['sum_d'] = df_sub_run['x_diff'] + df_sub_run['y_diff'] + df_sub_run['z_diff']
        
        df_sub_run['p_diff'] = abs(df_sub_run['p'].shift() - df_sub_run['p'])
        df_sub_run['r_diff'] = abs(df_sub_run['r'].shift() - df_sub_run['r'])
        df_sub_run['ya_diff'] = abs(df_sub_run['ya'].shift() - df_sub_run['ya'])
        df_sub_run['sum_r'] = df_sub_run['p_diff'] + df_sub_run['r_diff'] + df_sub_run['ya_diff']

        df_sub_run['reanalysis_FD'] = df_sub_run['sum_d'] + 50 * (np.pi /180) * df_sub_run['sum_r']
        
        appended_data.append(df_sub_run)
        
reanalysis_FD_abs = pd.concat(appended_data)
reanalysis_FD_abs

,x,y,z,p,r,ya,participant_ID,run,x_diff,y_diff,z_diff,sum_d,p_diff,r_diff,ya_diff,sum_r,reanalysis_FD
0,0.000000,0.040730,0.070119,-0.046352,0.000000,0.000000,102311,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000000,0.006030,0.047756,0.000000,0.000000,0.000000,102311,1,0.000000,0.034700,0.022363,0.057063,0.046352,0.000000,0.000000,0.046352,0.097513
2,0.000000,0.014362,0.078550,-0.021085,0.000000,0.000000,102311,1,0.000000,0.008332,0.030794,0.039126,0.021085,0.000000,0.000000,0.021085,0.057526
3,0.000007,0.089937,0.148111,-0.098491,0.000000,0.000000,102311,1,0.000007,0.075575,0.069561,0.145143,0.077406,0.000000,0.000000,0.077406,0.212692
4,0.000000,0.045510,0.111452,-0.077235,0.000000,0.000000,102311,1,0.000007,0.044427,0.036659,0.081093,0.021256,0.000000,0.000000,0.021256,0.099642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434940,0.085961,-0.149918,-0.415211,0.237434,-0.267972,0.056666,971160,4,0.022449,0.026170,0.024769,0.073388,0.047728,0.057984,0.009225,0.114937,0.173689
434941,0.075893,-0.066543,-0.401432,0.214286,-0.224370,0.069099,971160,4,0.010068,0.083375,0.013779,0.107222,0.023148,0.043602,0.012433,0.079183,0.176322
434942,0.071771,-0.027297,-0.384546,0.219557,-0.213255,0.060218,971160,4,0.004122,0.039246,0.016886,0.060254,0.005271,0.011115,0.008881,0.025267,0.082304
434943,0.074034,0.025249,-0.382854,0.219557,-0.222823,0.071161,971160,4,0.002263,0.052546,0.001692,0.056501,0.000000,0.009568,0.010943,0.020511,0.074400


In [45]:
#save only FD
reanalysis_FD_abs[['participant_ID', 'run', 'reanalysis_FD']]\
.to_csv("/Users/ziweizhang/Downloads/HCP37T_LL/output/motion/reanlysis_FD.csv",index=False)
